Search **TODO** to find out what needs to be done.

# 環境準備 Prepare enviroment

In [1]:
!pip install --upgrade datasets transformers accelerate evaluate


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [2]:
# 檢查GPU，Colab會自動分配GPU，Tesla T4以上比較好，不然建議終止工作階段重連來隨機更換GPU（重啟不會更換GPU）
# Check GPU, Colab will auto-allocate GPU, it is better to use Tesla T4 or a more powerful GPU, otherwise you should kill the runtime to switch GPU randomly
!nvidia-smi
import torch
from transformers import default_data_collator
from transformers import get_scheduler
from torch.optim import AdamW
from torch.utils.data.dataloader import DataLoader
import math
import os
import numpy as np
from tqdm.auto import tqdm

Tue Aug  8 15:48:51 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.103.01   Driver Version: 470.103.01   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| 35%   54C    P0    60W / 198W |      2MiB /  8119MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:17:00.0 Off |                  N/A |
|  0%   

2023-08-08 15:48:53.933980: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-08-08 15:48:54.578129: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:
2023-08-08 15:48:54.578203: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:
2023-08-08 15:48:54.578213: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are i

In [3]:
import transformers

print(transformers.__version__) # version check, 4.31.0 verified

4.31.0


# 微調預訓練模型 Fine-tune a Pre-trained Language Model

我們將會微調BERT等預訓練模型來進行QA任務，要注意的是，這個範例中的回答方式不是透過文本生成來回答問題，而是擷取給定段落中的文本片段來進行回答。目前，最常用的QA資料集為[SQuAD](https://rajpurkar.github.io/SQuAD-explorer/)，以下為SQuAD的範例：

We will fine-tune pre-trained models such as BERT for QA tasks. Note that in this example, instead of answering the question through text generation, the responses are extracted from text snippets in the given passage. Currently, the most commonly used QA dataset is [SQuAD](https://rajpurkar.github.io/SQuAD-explorer/), and the following is an example of SQuAD:

<img src="https://i.imgur.com/sOKTl1Z.jpg" width="500"/>

## 超參數設定 Hyperparameters setup

In [4]:
class arguments:
  def __init__(self,batch_size,model_checkpoint):
    self.preprocessing_num_workers=4
    self.max_seq_length=512
    self.pad_to_max_length=True
    self.model_name_or_path=model_checkpoint
    self.per_device_train_batch_size=batch_size
    self.per_device_eval_batch_size=batch_size
    self.learning_rate=3e-5
    self.weight_decay=0.01
    self.num_train_epochs=1
    self.max_train_steps=None
    self.gradient_accumulation_steps=1 # 設定需累積幾個batch的梯度再更新權重，可變相節省記憶體，1為不累積 Set to accumulate the gradient of several batch before updating the weights to save memory, 1 is not accumulate.
    self.lr_scheduler_type="linear"
    self.num_warmup_steps=0
    self.output_dir="output" # 如果不想存model和結果可以設置成None，大約佔400MB Set to None if you don't want to save the model and result, it takes about 400MB
    self.doc_stride=128 # 重疊的token長度 The overlapping token length
    self.n_best_size=20 # 選擇多少「答案」作為候選 How many "answers" to choose as candidates
    self.squad_v2 = False # 設定使用SQUAD版本 Switch between SQUAD v1 or v2
    self.null_score_diff_threshold=0.0 # 結果可能為無答案時選擇no answer的門檻 The threshold of choosing "no answer" when there is no answer.
    self.version_2_with_negative = False
    self.max_answer_length=30
    self.max_train_samples=4000 # 限制訓練資料大小來節省時間，正式訓練時請改回"None" Limit training data size to save time. Change to "None" when in actual training
    self.max_eval_samples=500 # 同上 As above
    self.overwrite_cache=True


# Download Model from HuggingFace Library. Supported model: BERT、ELECTRA、RoBERTa、DeBERTa. Others need to be confirmed. ref:https://huggingface.co/models
model_checkpoint = "bert-base-cased"
batch_size = 8 # Suggest gradient_accumulation_steps * batch_size >= 16
args = arguments(batch_size, model_checkpoint)

In [5]:
import os
# make sure output_dir exist
if args.output_dir is not None and not os.path.exists(args.output_dir):
  os.makedirs(args.output_dir)

## 下載資料集 Download dataset

這部分將使用[Datasets](https://github.com/huggingface/datasets) 提供的 `load_dataset` 來完成資料集準備。當然，直接從其他QA資料集的官方網站下載也可以。如果要用自己的json、csv格式的dataset，load_dataset也可以完成。請看官方文檔[Datasets documentation](https://huggingface.co/docs/datasets/loading_datasets.html#from-local-files)。

This part will use `load_dataset` provided by [Datasets](https://github.com/huggingface/datasets) to complete the dataset preparation. Of course, it is possible to download it directly from the official website of other QA datasets. If you want to use your own dataset in json, csv format, load_dataset can also be done. Please see the official documentation [Datasets documentation](https://huggingface.co/docs/datasets/loading_datasets.html#from-local-files).


In [6]:
from datasets import load_dataset

datasets = load_dataset("squad_v2" if args.squad_v2 else "squad")

In [7]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

print("datasets structure:\n")
pp.pprint(datasets)
print("\n\ntrain data：\n")
pp.pprint(datasets['train'][0])
print("\n\nvalidation data：\n")
pp.pprint(datasets['validation'][0])

datasets structure:

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})


train data：

{   'answers': {'answer_start': [515], 'text': ['Saint Bernadette Soubirous']},
    'context': 'Architecturally, the school has a Catholic character. Atop the '
               "Main Building's gold dome is a golden statue of the Virgin "
               'Mary. Immediately in front of the Main Building and facing it, '
               'is a copper statue of Christ with arms upraised with the '
               'legend "Venite Ad Me Omnes". Next to the Main Building is the '
               'Basilica of the Sacred Heart. Immediately behind the basilica '
               'is the Grotto, a Marian place of prayer and reflection. It is '
               'a replica of the grotto at Lourdes, France where t

## 資料前處理 Preprocessing

將文本轉為embedding，通常Transformers提供的 Tokenizer 可以自動將文本Tokenize並轉換為model可以讀取的形式。BERT的輸入格式參考下方圖片：  

To convert text to embedding, usually the Tokenizer provided by Transformers can automatically tokenize the text and convert it into a form that can be read by the model, refer to the image below for the input format of BERT:

<img src="https://i.imgur.com/xQIkHWu.png" width="800" />

In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_path)

In [9]:
# 檢視tokenizer輸出 Quick view about tokenizer output
query = "Who love IISR?"
context = "IISR is good. Everyone loves it."
token = tokenizer(query, context)
pp.pprint(token)
print(tokenizer.convert_ids_to_tokens(token["input_ids"]))

{   'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    'input_ids': [   101,
                     2627,
                     1567,
                     1563,
                     16396,
                     136,
                     102,
                     1563,
                     16396,
                     1110,
                     1363,
                     119,
                     6064,
                     7871,
                     1122,
                     119,
                     102],
    'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
['[CLS]', 'Who', 'love', 'II', '##SR', '?', '[SEP]', 'II', '##SR', 'is', 'good', '.', 'Everyone', 'loves', 'it', '.', '[SEP]']


In [10]:
# 主要處理的問題是部分passage可能超過設定的max_seq_length，因此需要對文章分段，並調整答案的index
# The main problem is some passage length may be longer than max_seq_length. So it needs segmentation and re-indexing
# 注意，為了簡化流程在這我們只切分兩資料集，一份訓練一份評估
# Note that in order to simplify the process here we only cut it into two datasets, one for training and one for evaluation

from dataset_preprocess import QAdataset
SQuAD_dataset = QAdataset(datasets, tokenizer, args)

train_dataset = SQuAD_dataset.generate_train_dataset()
eval_dataset = SQuAD_dataset.generate_eval_dataset()

print("train_dataset:")
pp.pprint(train_dataset)

print("\n\neval_dataset:")
pp.pprint(eval_dataset)

Running tokenizer on train dataset (num_proc=4):   0%|          | 0/4000 [00:00<?, ? examples/s]

Running tokenizer on validation dataset (num_proc=4):   0%|          | 0/500 [00:00<?, ? examples/s]

train_dataset:
Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 4000
})


eval_dataset:
Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'example_id'],
    num_rows: 500
})


## 建構模型輸出層 Construct output layer

In [11]:
from torch import nn
from torch.nn import CrossEntropyLoss

class QuestionAnsweringModelOutput: # The format of output
    def __init__(self, loss, start_logits, end_logits):
      self.loss = loss
      self.start_logits = start_logits
      self.end_logits = end_logits

class OutputQA(nn.Module): # Only output layer

    def __init__(self,hidden_dim):
        super(OutputQA, self).__init__()
        self.num_labels = 2
        self.qa_outputs_lin = nn.Linear(hidden_dim, 2)# TODO # 提示，使用linear即可 (Use linear layer)

    def forward(
        self,
        hidden_state,
        start_positions=None,
        end_positions=None
    ):

        logits = self.qa_outputs_lin(hidden_state)# TODO

        start_logits, end_logits = torch.split(logits, 1, 2)# TODO # 提示，需要使用到torch.split()方法 (This need torch.split() func)

        start_logits = start_logits.squeeze(-1).contiguous()
        end_logits = end_logits.squeeze(-1).contiguous()



        total_loss = None
        if start_positions is not None and end_positions is not None:
            # If we are on multi-GPU, split add a dimension
            if len(start_positions.size()) > 1:
                start_positions = start_positions.squeeze(-1)
            if len(end_positions.size()) > 1:
                end_positions = end_positions.squeeze(-1)

            # 若start/end 位置超出inputs範圍，需要調整在範圍之內
            # Adjust the start/end position, if they out of the range of inputs
            max_index = start_logits.size(1)
            start_positions = torch.clamp(start_positions, min=0, max=max_index)# TODO # 提示，使用torch.clamp()調整數值範圍 (Use torch.clamp() to adjust the range of values)
            end_positions =   torch.clamp(end_positions, min=0, max=max_index)# TODO # 提示，使用torch.clamp()調整數值範圍 (Use torch.clamp() to adjust the range of values)

            # loss 計算，提示：建議使用CrossEntropyLoss
            # Calculate the loss, Suggest use CrossEntropyLoss

            loss_fct = CrossEntropyLoss() # TODO
            start_loss = loss_fct(start_logits, start_positions)# TODO
            end_loss =   loss_fct(end_logits, end_positions)# TODO
            total_loss = start_loss + end_loss # TODO # 簡單相加即可 Simply add up


        return QuestionAnsweringModelOutput(
            loss=total_loss,
            start_logits=start_logits,
            end_logits=end_logits
        )


## 微調模型 Fine-tune model

可以使用Transformers預設好的`AutoModelForQuestionAnswering`進行預訓練，但在這邊我們自己定義。  
Pre-training can be done using Transformers' preset `AutoModelForQuestionAnswering`, but we define it ourselves here.

In [12]:
# Load pre-train model
from transformers import RobertaModel, ElectraModel, DebertaModel, BertModel, AutoModel

def select_model(model_name):
  model_types = {
    "roberta": RobertaModel,
    "electra": ElectraModel,
    "deberta": DebertaModel,
    "bert": BertModel
  }
  for model_type in model_types.keys():
    if model_type in model_name:
      return model_types[model_type].from_pretrained(model_name)
  print("warning: Using AutoModel but not sure about the type of model_name_or_path.")
  return AutoModel.from_pretrained(args.model_name_or_path)

model = select_model(args.model_name_or_path)
output_layer = OutputQA(model.config.hidden_size)

In [13]:
# Dataloader

# default_data_collator會將數筆資料堆疊在一起形成batch
# default_data_collator will stack rows of data together to become a batch
data_collator = default_data_collator

# 通常會將訓練資料打亂以提升通用性，避免過擬合
# Often the training data will be shuffled to enhance generalizability and to avoid overfitting
train_dataloader = DataLoader(train_dataset, shuffle=True, collate_fn=data_collator, batch_size=args.per_device_train_batch_size)

# 通常會將驗證資料不打亂避免造成分數計算的不一致
# Often the validation data is not shuffled to avoid inconsistencies in the score calculation
eval_dataset_for_model = eval_dataset.remove_columns(["example_id", "offset_mapping"])
eval_dataloader = DataLoader(eval_dataset_for_model, shuffle=False, collate_fn=data_collator, batch_size=args.per_device_eval_batch_size)

In [14]:
# Optimizer
# 輸出層的學習率通常要設置大於原Bert的學習率。
# 另外Bert權重分兩組，一個有weight decay，另一組不設。
# Weight decay是在loss中加入懲罰，用來避免過擬合。
# The learning rate of the output layer is usually set larger than the original Bert's learning rate.
# In addition, the Bert weights are divided into two groups, one with weight decay and the other without.
# The weight decay is a penalty added to the loss to avoid overfitting.
no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in output_layer.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": args.weight_decay,
        "lr": args.learning_rate*10, # 10 times larger
    },
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": args.weight_decay,
    },
    {
        "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    }
]
optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate)

In [15]:
from accelerate import Accelerator
# 用`accelerate`自動切換設備/多線程，跟寫.cuda()或.to(device)效果相同，可參考 https://pypi.org/project/accelerate/
# Use `accelerate` to switch device/multi-threading automatically, same effect as writing .cuda() or .to(device), see https://pypi.org/project/accelerate/
accelerator = Accelerator()
print(accelerator.state)

# 需注意倒傳遞時要用`accelerator.backward(loss)`取代`loss.backward()`
# Be careful to replace `accelerator.backward(loss)` with `loss.backward()` when back propagation
output_layer, model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    output_layer, model, optimizer, train_dataloader, eval_dataloader
)

Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: no



In [16]:
# 根據步數設定學習率調整策略，可以調整args.lr_scheduler_type為其他策略，例如cosine
# Set the learning rate adjustment strategy based on the number of steps, you can adjust args.lr_scheduler_type to other strategies, such as cosine
num_update_steps_per_epoch = math.ceil(len(train_dataloader) / args.gradient_accumulation_steps) # 計算總訓練步數 Total training step

if args.max_train_steps is None:
    args.max_train_steps = args.num_train_epochs * num_update_steps_per_epoch
else:
    args.num_train_epochs = math.ceil(args.max_train_steps / num_update_steps_per_epoch)

lr_scheduler = get_scheduler(
    name=args.lr_scheduler_type, # default: linear
    optimizer=optimizer,
    num_warmup_steps=args.num_warmup_steps, # warm up steps is usually 1/10 of the total steps, we default 0 here
    num_training_steps=args.max_train_steps,
)

In [17]:
# Start training
total_batch_size = args.per_device_train_batch_size * accelerator.num_processes * args.gradient_accumulation_steps

print("***** Running training *****")
print(f"  Num examples = {len(train_dataset)}")
print(f"  Num Epochs = {args.num_train_epochs}")
print(f"  Instantaneous batch size per device = {args.per_device_train_batch_size}")
print(f"  Total train batch size (w. parallel, distributed & accumulation) = {total_batch_size}")
print(f"  Gradient Accumulation steps = {args.gradient_accumulation_steps}")
print(f"  Total optimization steps = {args.max_train_steps}")

# Setup progress bar. The "disable" parameter is to limit only show one progress bar when using multiple GPUs, it doesn't matter in Colab
# 進度條設定，disable部分是為了規定多個GPU時只顯示一個進度條，在Colab中有沒有都不影響
progress_bar = tqdm(range(args.max_train_steps), disable=not accelerator.is_local_main_process)

completed_steps = 0
for epoch in range(args.num_train_epochs):
    model.train()
    output_layer.train()
    for step, batch in enumerate(train_dataloader):
        # batch裡包含了訓練用的word index, mask, segment以及算loss用的label，只將需要的資料傳進model
        # batch contains the word index, mask, segment for training and label for loss calculation, only the required data is passed into the model.
        input = {key: value for key, value in batch.items() if key not in ['start_positions','end_positions']}
        encoding = model(**input)
        outputs = output_layer(encoding['last_hidden_state'], batch['start_positions'], batch['end_positions']) # TODO
        loss = outputs.loss / args.gradient_accumulation_steps
        accelerator.backward(loss)
        if step % args.gradient_accumulation_steps == 0 or step == len(train_dataloader) - 1:
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            progress_bar.update(1)
            completed_steps += 1

        if completed_steps >= args.max_train_steps:
            break

***** Running training *****
  Num examples = 4000
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 500


  0%|          | 0/500 [00:00<?, ?it/s]

# 評估模型 Evalation

### 評估方法說明

SQuAD使用Exact Match (EM)和F1 score計算分數。對每個「問題+答案」對計算並在最後平均而得。當一個問題可能有多個正確答案時，計算所有可能的正確答案的最大分數。

1.   EM：對於每個「問題+答案」對，如果模型預測的詞句與（其中一個）真實答案的詞句完全匹配，EM=1，否則EM=0。嚴格的全有或全無指標；偏離一個詞句就會得到0分。
2.   F1：透過計算預測和真實答案相同token數來得出分數：
  *   Precision = 相同token數量與預測總token數的比例
  *   Recall = 相同token數量與真實答案總token數的比例
  *   F1 = 2 \* (Precision \* Recall) / (Precision + Recall)

SQuAD uses Exact Match (EM) and F1 score to calculate scores. Each "question + answer" pair is calculated and averaged at the end. When a question may have more than one correct answer, the maximum score of all possible correct answers is calculated.

1. EM: For each "question-answer" pair, EM = 1 if the model's predicted phrase exactly matches (one of) the true answer's phrase, otherwise EM = 0. Strictly an all-or-nothing metric; a deviation of one phrase results in a score of 0. 2.
2. F1: Score is obtained by calculating the number of tokens that are the same in the prediction and ground truth:
  * Precision = the ratio of the number of same tokens to total tokens in the prediction.
  * Recall = the ratio of the number of same tokens to total tokens in the ground truth.
  * F1 = 2 \* (Precision \* Recall) / (Precision + Recall)



In [18]:
import evaluate
# 讀取預先設計好的評估方法 Load pre-defined metric
metric = evaluate.load("squad_v2" if args.squad_v2 else "squad")
print(metric.description)


This metric wrap the official scoring script for version 1 of the Stanford Question Answering Dataset (SQuAD).

Stanford Question Answering Dataset (SQuAD) is a reading comprehension dataset, consisting of questions posed by
crowdworkers on a set of Wikipedia articles, where the answer to every question is a segment of text, or span,
from the corresponding reading passage, or the question might be unanswerable.



In [19]:
# Evalation
from utils_qa import post_processing_function, create_and_fill_np_array


print("***** Running Evaluation *****")
print(f"  Num examples = {len(eval_dataset)}")
print(f"  Batch size = {args.per_device_eval_batch_size}")
print(f"  squad_version = {metric.name}")


progress_bar = tqdm(range(len(eval_dataloader)), disable=not accelerator.is_local_main_process)

model.eval()
output_layer.eval()

all_start_logits = []
all_end_logits = []
for step, batch in enumerate(eval_dataloader):
    with torch.no_grad():
        encoding = model(**batch)
        outputs = output_layer(encoding['last_hidden_state'], batch['start_positions'], batch['end_positions']) # TODO
        start_logits = outputs.start_logits
        end_logits = outputs.end_logits

        if not args.pad_to_max_length:  # 統一所有進程的padding長度才能使用accelerator.gather() Align the padding lengths of all processes in order to use accelerator.gather()
            start_logits = accelerator.pad_across_processes(start_logits, dim=1, pad_index=-100) # pad tensor across processes to max length
            end_logits = accelerator.pad_across_processes(end_logits, dim=1, pad_index=-100)

        all_start_logits.append(accelerator.gather(start_logits).cpu().numpy()) # 分散式運算時用於集合預測結果 Aggregate prediction results when distributed computing
        all_end_logits.append(accelerator.gather(end_logits).cpu().numpy())
    progress_bar.update(1)

max_len = max([x.shape[1] for x in all_start_logits])

# concatenate array
all_start_logits = create_and_fill_np_array(all_start_logits, eval_dataset, max_len)
all_end_logits = create_and_fill_np_array(all_end_logits, eval_dataset, max_len)


outputs_numpy = (all_start_logits, all_end_logits) # 預測結果 prediction result
prediction = post_processing_function(datasets['validation'], eval_dataset, outputs_numpy,args) # 後處理 post-processing , ref: https://reurl.cc/ZW3XjM
eval_metric = metric.compute(predictions=prediction.predictions, references=prediction.label_ids)
print(f"Evaluation metrics: {eval_metric}")

SyntaxError: invalid syntax (2282032042.py, line 21)

In [ ]:
# 觀察直接選擇start_logits和end_logits各自最大值的結果與經過post_processing_function處理過後的結果之間的差異
# Check the difference between the result of choosing the maximum values of start_logits and end_logits directly and the result after post_processing_function.
for qs_id in range(10):
  start_idx = outputs_numpy[0][qs_id].argmax()
  end_idx = outputs_numpy[1][qs_id].argmax()
  print(f"{start_idx}-{end_idx}")
  print(" ".join(tokenizer.convert_ids_to_tokens(eval_dataset['input_ids'][qs_id][start_idx:end_idx+1])))
  print(prediction.predictions[qs_id]['prediction_text'])
  print('========')

In [ ]:
# 人工查看，比較預測結果與和答案
# Manually check and compare predictions with ground truth

for i in range(0, 500, 50):
  print("----Question----")
  pp.pprint(datasets['validation'][i])
  print("\n----Prediction----")
  pp.pprint(prediction.predictions[i])
  print("="*70+"\n")

In [ ]:
# 儲存微調好的模型
# Save the fine-tuned model
if args.output_dir is not None:
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(args.output_dir, save_function=accelerator.save)